In [1]:
import pandas as pd
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain.agents.agent_types import AgentType
import httpx
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain_experimental.sql.base import SQLDatabaseChain
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_core.messages import AIMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_community.agent_toolkits.sql.prompt import SQL_FUNCTIONS_SUFFIX
from langchain.agents import AgentExecutor, create_openai_tools_agent, create_openai_functions_agent
import sqlite3

In [2]:
API_KEY = ""

In [3]:
csv_file = "../data/hospitals_emergency_data.csv"
df = pd.read_csv(csv_file)

In [4]:
conn = sqlite3.connect("../src/emergency.db")

In [5]:
cursor = conn.cursor()

In [6]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS emergency_directory (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        zip_code TEXT NOT NULL,
        hospital_name TEXT NOT NULL,
        ambulance_available TEXT NOT NULL
    )
""")

In [7]:
df.to_sql("emergency_directory", conn, if_exists="replace", index=False)

31

In [8]:
cursor.execute("SELECT * FROM emergency_directory LIMIT 5;")
rows = cursor.fetchall()
for row in rows:
    print(row)

(10001, 'Crestwood Clinic', 'No')
(10001, 'Wellness Medical Institute', 'No')
(10002, 'Greenwood Healthcare', 'Yes')
(10002, 'Riverbend Medical Center', 'No')
(10002, 'City Care Hospital', 'No')


In [9]:
conn.close()

In [16]:
memory = ConversationBufferMemory(memory_key="history")
llm = ChatOpenAI(openai_api_key=API_KEY, temperature=0, model="gpt-4o-2024-08-06", max_tokens=500, http_client = httpx.Client(verify=False))

In [17]:
db = SQLDatabase.from_uri("sqlite:///emergency.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
context = toolkit.get_context()
tools = toolkit.get_tools()

messages = [
            HumanMessagePromptTemplate.from_template("{input}"),
            AIMessage(content=SQL_FUNCTIONS_SUFFIX + "\nUse like operator with lowercase when matching a name\nWhen user is asking to book available ambulance, delete the corresponding hospital row from the table\n"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
            ]
prompt = ChatPromptTemplate.from_messages(messages)
prompt = prompt.partial(**context)
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
            agent=agent,
            tools=toolkit.get_tools(),
            verbose=True,
        )

In [18]:
user_input = 'any ambulance available at 94404'

In [19]:
agent_executor.invoke({"input": user_input})['output']



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with `{}`


emergency_directory
Invoking: `sql_db_schema` with `{'table_names': 'emergency_directory'}`



CREATE TABLE emergency_directory (
	"Zip Code" INTEGER, 
	"Hospital Name" TEXT, 
	"Ambulance Available" TEXT
)

/*
3 rows from emergency_directory table:
Zip Code	Hospital Name	Ambulance Available
10001	Crestwood Clinic	No
10001	Wellness Medical Institute	No
10002	Greenwood Healthcare	Yes
*/
Invoking: `sql_db_query` with `{'query': 'SELECT "Hospital Name" FROM emergency_directory WHERE "Zip Code" = 94404 AND "Ambulance Available" = \'Yes\''}`


[('Clearview Hospital',)]Yes, there is an ambulance available at Clearview Hospital in the 94404 area.

> Finished chain.


'Yes, there is an ambulance available at Clearview Hospital in the 94404 area.'

In [20]:
agent_executor.invoke({"input": "book the ambuance near 94404 zipcode"})['output']



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with `{}`


emergency_directory
Invoking: `sql_db_schema` with `{'table_names': 'emergency_directory'}`



CREATE TABLE emergency_directory (
	"Zip Code" INTEGER, 
	"Hospital Name" TEXT, 
	"Ambulance Available" TEXT
)

/*
3 rows from emergency_directory table:
Zip Code	Hospital Name	Ambulance Available
10001	Crestwood Clinic	No
10001	Wellness Medical Institute	No
10002	Greenwood Healthcare	Yes
*/
Invoking: `sql_db_query_checker` with `{'query': 'SELECT "Hospital Name" FROM emergency_directory WHERE "Zip Code" = 94404 AND "Ambulance Available" = \'Yes\' LIMIT 1;'}`


```sql
SELECT "Hospital Name" FROM emergency_directory WHERE "Zip Code" = 94404 AND "Ambulance Available" = 'Yes' LIMIT 1;
```
Invoking: `sql_db_query` with `{'query': 'SELECT "Hospital Name" FROM emergency_directory WHERE "Zip Code" = 94404 AND "Ambulance Available" = \'Yes\' LIMIT 1;'}`


[('Clearview Hospital',)]
Invoking: `sql_db_query_checker` with

'The ambulance from Clearview Hospital near the 94404 zip code has been successfully booked for you.'